# Fruits-360 - Transfer Learning using Keras and ResNet-50

https://www.kaggle.com/amadeus1996/fruits-360-transfer-learning-using-keras


* This notebook is a brief application of transfer learning on the Fruits-360 [Dataset](https://www.kaggle.com/moltean/fruits). 
* This data set consists of 42345 images of 64 fruits.
* We compare the Transfer learning approach to the regular approach.
* Accuracy of 98.44% is achieved within 2 epochs.


### **Contents:**

*  **1. Brief Explanation of Transfer Learning**
*  **2. Transfer Learning using Kaggle Kernels**
*  **3. Reading and Visualizing the Data**   
*  **4. Building and Compiling the Models**    
*  **5. Training and Validating the Pretrained Model** 
*  **6. Training and Validating Vanilla Model**
*  **7. Comparison between Pretrained Models and Vanilla Model**

In [1]:
import os
from os import listdir, makedirs
from os.path import join, exists, expanduser

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import tensorflow as tf
# Any results you write to the current directory are saved as output.

## 1. Transfer Learning

In transfer learning, we first train a base network on a base dataset and task, and then we repurpose the learned features, or transfer them, to a second target network to be trained on a target dataset and task. This process will tend to work if the features are general, meaning suitable to both base and target tasks, instead of specific to the base task.

Lisa Torrey and Jude Shavlik in their chapter on transfer learning describe three possible benefits to look for when using transfer learning:

* Higher start. The initial skill (before refining the model) on the source model is higher than it otherwise would be.
* Higher slope. The rate of improvement of skill during training of the source model is steeper than it otherwise would be.
* Higher asymptote. The converged skill of the trained model is better than it otherwise would be.

<center><img src="https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2017/09/Three-ways-in-which-transfer-might-improve-learning.png"></center>


Basically, we take a pre-trained model (the weights and parameters of a network that has been trained on a large dataset by somebody else) and “fine-tune” the model with our own dataset. The idea is that this pre-trained model will either provide the initialized weights leading to a faster convergence or it will act as a fixed feature extractor for the task of interest.



These two major transfer learning scenarios look as follows:

* Finetuning the convnet: Instead of random initializaion, we initialize the network with a pretrained network, like the one that has been trained on a large dataset like imagenet 1000. Rest of the training looks as usual. In this scenario the entire network needs to be retrained on the dataset of our interest

* ConvNet as fixed feature extractor: Here, we will freeze the weights for all of the network except that of the final fully connected layer. This last fully connected layer is replaced with a new one with random weights and only this layer is trained.

In this notebook we will demonstrate the first scenario.


## 2. Transfer Learning using Kaggle Kernels

### Using the Keras Pretrained Models dataset
Kaggle Kernels cannot use a network connection to download pretrained keras models. This [Dataset](https://www.kaggle.com/moltean/fruits) helps us to use our favorite pretrained models in the Kaggle Kernel environment.

All we have to do is to copy the pretrained models to the cache directory (~/.keras/models) where keras is looking for them.

In [2]:
'''
cache_dir = expanduser(join('~', '.keras'))
if not exists(cache_dir):
    makedirs(cache_dir)
models_dir = join(cache_dir, 'models')
if not exists(models_dir):
    makedirs(models_dir)
    
!cp ../input/keras-pretrained-models/*notop* ~/.keras/models/
!cp ../input/keras-pretrained-models/imagenet_class_index.json ~/.keras/models/
!cp ../input/keras-pretrained-models/resnet50* ~/.keras/models/

print("Available Pretrained Models:\n")
!ls ~/.keras/models
'''

'\ncache_dir = expanduser(join(\'~\', \'.keras\'))\nif not exists(cache_dir):\n    makedirs(cache_dir)\nmodels_dir = join(cache_dir, \'models\')\nif not exists(models_dir):\n    makedirs(models_dir)\n    \n!cp ../input/keras-pretrained-models/*notop* ~/.keras/models/\n!cp ../input/keras-pretrained-models/imagenet_class_index.json ~/.keras/models/\n!cp ../input/keras-pretrained-models/resnet50* ~/.keras/models/\n\nprint("Available Pretrained Models:\n")\n!ls ~/.keras/models\n'

## 3. Reading and Visualizing the Data
### Reading the Data

Like the rest of Keras, the image augmentation API is simple and powerful. We will use the **ImageDataGenerator** to fetch data and feed it to our network

Keras provides the **ImageDataGenerator** class that defines the configuration for image data preparation and augmentation. Rather than performing the operations on your entire image dataset in memory, the API is designed to be iterated by the deep learning model fitting process, creating augmented image data for you just-in-time. This reduces your memory overhead, but adds some additional time cost during model training.

The data generator itself is in fact an iterator, returning batches of image samples from the directory when requested. We can configure the batch size and prepare the data generator and get batches of images by calling the **flow_from_directory()** function.

In [1]:
# Configuramos Drive
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [4]:
# dimensions of our images.
img_width, img_height = 64, 64 # we set the img_width and img_height according to the pretrained models we are

# going to use. The input size for ResNet-50 is 224 by 224 by 3.
train_data_dir = '/content/gdrive/MyDrive/Archivos/Datasets/fruits/Training/'
validation_data_dir = '/content/gdrive/MyDrive/Archivos/Datasets/fruits/Validation/'

nb_train_samples = 6404
nb_validation_samples = 2134
batch_size = 16

In [5]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 6404 images belonging to 13 classes.
Found 2134 images belonging to 13 classes.


### Visualizing the Data

In [6]:
import pandas as pd
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [7]:
training_data = pd.DataFrame(train_generator.classes, columns=['classes'])
testing_data = pd.DataFrame(validation_generator.classes, columns=['classes'])

In [8]:
def create_stack_bar_data(col, df):
    aggregated = df[col].value_counts().sort_index()
    x_values = aggregated.index.tolist()
    y_values = aggregated.values.tolist()
    return x_values, y_values

In [9]:
x1, y1 = create_stack_bar_data('classes', training_data)
x1 = list(train_generator.class_indices.keys())

trace1 = go.Bar(x=x1, y=y1, opacity=0.75, name="Class Count")
layout = dict(height=400, width=1200, title='Class Distribution in Training Data', legend=dict(orientation="h"), 
                yaxis = dict(title = 'Class Count'))
fig = go.Figure(data=[trace1], layout=layout);
iplot(fig);

In [10]:
x1, y1 = create_stack_bar_data('classes', testing_data)
x1 = list(validation_generator.class_indices.keys())

trace1 = go.Bar(x=x1, y=y1, opacity=0.75, name="Class Count")
layout = dict(height=400, width=1100, title='Class Distribution in Validation Data', legend=dict(orientation="h"), 
                yaxis = dict(title = 'Class Count'))
fig = go.Figure(data=[trace1], layout=layout);
iplot(fig);

> - As we can see, all the classes are extremely well-balanced in training as well as the validation.

## 4. Building and Compiling the Model
### Building the Models

Here, we load the ResNet-50 with the ImageNet weights. We remove the top so that we can add our own layer according to the number of our classes. We then add our own layers to complete the model architecture.

* ### Building Pretrained Model

In [11]:
#import inception with pre-trained weights. do not include fully #connected layers
inception_base = applications.ResNet50(input_shape=(img_width, img_height, 3),
                                       weights='imagenet',
                                       include_top=False)

# add a global spatial average pooling layer
x = inception_base.output
x = GlobalAveragePooling2D()(x)
# add a fully-connected layer
x = Dense(512, activation='relu')(x)
# and a fully connected output/classification layer
predictions = Dense(13, activation='softmax')(x)
# create the full network so we can train on it
inception_transfer = Model(inputs=inception_base.input, outputs=predictions)

94773248/94765736 [==============================] - 1s 0us/step


*  ### Building the Vanilla Model

In [12]:
#import inception without pre-trained weights. do not include fully #connected layers
inception_base_vanilla = applications.ResNet50(input_shape=(img_width, img_height,3),
                                               weights=None,
                                               include_top=False)

# add a global spatial average pooling layer
x = inception_base_vanilla.output
x = GlobalAveragePooling2D()(x)
# add a fully-connected layer
x = Dense(512, activation='relu')(x)
# and a fully connected output/classification layer
predictions = Dense(13, activation='softmax')(x)
# create the full network so we can train on it
inception_transfer_vanilla = Model(inputs=inception_base_vanilla.input, outputs=predictions)

### Compiling the Models
We set the loss function, the optimization algorithm to be used and metrics to be calculated at the end of each epoch.

In [13]:
inception_transfer.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

inception_transfer_vanilla.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [14]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13427052428174369617
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11154422528
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16549207559540194328
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


## 5. Training and Validating the Pretrained Model

We use the **fit_generator()** function because we are using object of the **ImageDataGenerator** class to fetch data.

In [15]:
import tensorflow as tf
with tf.device("/device:GPU:0"):
    history_pretrained = inception_transfer.fit(
    train_generator,
    epochs=5, shuffle = True, verbose = 1, validation_data = validation_generator)

Epoch 1/5
401/401 [==============================] - 5738s 14s/step - loss: 1.8405 - accuracy: 0.4472 - val_loss: 86.2397 - val_accuracy: 0.0712
Epoch 2/5
401/401 [==============================] - 52s 129ms/step - loss: 0.3234 - accuracy: 0.9186 - val_loss: 2.2406 - val_accuracy: 0.3055
Epoch 3/5
401/401 [==============================] - 52s 129ms/step - loss: 0.1667 - accuracy: 0.9558 - val_loss: 0.1218 - val_accuracy: 0.9611
Epoch 4/5
401/401 [==============================] - 52s 129ms/step - loss: 0.0965 - accuracy: 0.9767 - val_loss: 0.0450 - val_accuracy: 0.9911
Epoch 5/5
401/401 [==============================] - 52s 129ms/step - loss: 0.0770 - accuracy: 0.9818 - val_loss: 0.0459 - val_accuracy: 0.9817


In [ ]:
with tf.device("/device:GPU:0"):
    history_vanilla = inception_transfer_vanilla.fit(
    train_generator,
    epochs=5, shuffle = True, verbose = 1, validation_data = validation_generator)

Epoch 1/5
401/401 [==============================] - 59s 133ms/step - loss: 2.5912 - accuracy: 0.1757 - val_loss: 1.4781 - val_accuracy: 0.5328
Epoch 2/5
401/401 [==============================] - 52s 130ms/step - loss: 1.1234 - accuracy: 0.6099 - val_loss: 0.7241 - val_accuracy: 0.7245
Epoch 3/5
401/401 [==============================] - 52s 130ms/step - loss: 0.7432 - accuracy: 0.7504 - val_loss: 0.6624 - val_accuracy: 0.7666
Epoch 4/5
111/401 [=======>......................] - ETA: 34s - loss: 0.5161 - accuracy: 0.8245

In [ ]:
import matplotlib.pyplot as plt
# summarize history for accuracy
plt.plot(history_pretrained.history['val_acc'])
plt.plot(history_vanilla.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Pretrained', 'Vanilla'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history_pretrained.history['val_loss'])
plt.plot(history_vanilla.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Pretrained', 'Vanilla'], loc='upper left')
plt.show()